In [14]:
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from sklearn.model_selection import KFold, train_test_split
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
import cv2
import torch

In [19]:
# Set paths and parameters
solvable_path = Path('./17')
unsolvable_path = Path('./17_unsolvable_same_wall_count')
max_images = 20000  # Total number of images to load

# Number of images to load per class
images_per_class = max_images // 2

images = []  # List to store image data
labels = []  # List to store labels

def load_and_label_images(folder_path, label, max_images, reverse_order=False):
    # sort png files
    img_paths = sorted(folder_path.glob('*.png'))
    #random.shuffle(img_paths)
    
    # If reverse_order is True reverse the sorted list
    if reverse_order:
        img_paths = img_paths[::-1]
    
    count = 0
    for img_path in img_paths:
        if count >= max_images:
            break
        img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        images.append(img_array)
        labels.append(label)
        count += 1

load_and_label_images(solvable_path, 1, images_per_class, reverse_order=False) 
load_and_label_images(unsolvable_path, 0, images_per_class, reverse_order=False) 

In [20]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize images to [0,1]
images = images / 255.0

# Split 
X_train_val, X_test, y_train_val, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)

# Define a function to create and compile the model
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.03), input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Cross validation on the training set
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
acc_per_fold = []
loss_per_fold = []

for train_index, val_index in kfold.split(X_train_val, y_train_val):
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    y_train, y_val = y_train_val[train_index], y_train_val[val_index]

    # new model for each run
    model = create_model((images.shape[1], images.shape[2], 1))

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=4,
        batch_size=16,
        validation_data=(X_val, y_val),
        verbose=1
    )

    # model results for this fold
    scores = model.evaluate(X_val, y_val, verbose=0)
    print(f'Fold {fold_no} - Validation Loss: {scores[0]} - Validation Accuracy: {scores[1]}')
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])

    fold_no += 1

# Results from cross-validation
print('Score per fold:')
for i in range(len(acc_per_fold)):
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

final_model = create_model((images.shape[1], images.shape[2], 1))
final_model.fit(X_train_val, y_train_val, epochs=4, batch_size=16, verbose=1)

# Test set
test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


Epoch 1/4


C:\Users\shade_of_ox\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4973 - loss: 0.7101 - val_accuracy: 0.4938 - val_loss: 0.6934
Epoch 2/4
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4938 - loss: 0.6934 - val_accuracy: 0.4938 - val_loss: 0.6932
Epoch 3/4
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5066 - loss: 0.6932 - val_accuracy: 0.5063 - val_loss: 0.6931
Epoch 4/4
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4962 - loss: 0.6932 - val_accuracy: 0.4938 - val_loss: 0.6932
Fold 1 - Validation Loss: 0.6932157278060913 - Validation Accuracy: 0.4937500059604645
Epoch 1/4
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4978 - loss: 0.7111 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/4
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4964 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/4
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5002 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/4
800/800 ━━━━━━━━━━━━━━━━